# 이항모델 - 이항분포 옵션 가격 결정 모형(CRR model)

In [1]:
import numpy as np
import pandas as pd
import math

## CRR model

Assumption: Call option<br>
S0 : 현재주가<br>
V : 주가의 기간별 변동성<br>
T : 이항분포가 발생하는 전체 기간<br>
dt : 이항분포가 1회 발생하는 단위기간(본 모델에서는 편의상 1로 설정)<br>
Rf : 무위험 이자율 - 이산복리<br>
K : 옵션의 행사가격

In [2]:
S0 = 100
V = 0.3
T = 5
dt = 1
Rf = 0.05
K = 100

u : 주가상승배수<br>
d : 주가하락배수<br>
P : 위험중립확률<br>
<br>
$S_0 = [P \times u \times S_0 + (1-P) \times d \times e^{(-Rf)}]$

In [3]:
u = math.exp(V * math.sqrt(dt))
d = 1/u
P = (math.exp(Rf * dt) - d) / (u-d)
print(u, d, P)

1.3498588075760032 0.7408182206817179 0.5097408651817704


In [4]:
S_tree = pd.DataFrame(np.zeros((T+1) * (T+1)).reshape(T+1, T+1))

In [5]:
S_tree

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


### 주가 트리

In [6]:
for node in range(T+1):
    for time in range(T+1):
        if time >= node:
            S_tree.loc[node, time] = S0 * math.pow(u, (time - node)) * math.pow(d, node)
S_tree

,0,1,2,3,4,5
0,100.0,134.985881,182.211880,245.960311,332.011692,448.168907
1,0.0,74.081822,100.000000,134.985881,182.211880,245.960311
2,0.0,0.000000,54.881164,74.081822,100.000000,134.985881
3,0.0,0.000000,0.000000,40.656966,54.881164,74.081822
4,0.0,0.000000,0.000000,0.000000,30.119421,40.656966
5,0.0,0.000000,0.000000,0.000000,0.000000,22.313016


### 시점별 노드별 옵션의 기대값

In [7]:
C_K_tree = pd.DataFrame(np.zeros((T+1)*(T+1)).reshape(T+1, T+1))
for node in range(T+1):
    for time in range(T+1):
        if time >= node:
            C_K_tree.loc[node, time] = max(S_tree.loc[node, time] - K, 0)
C_K_tree

,0,1,2,3,4,5
0,0.0,34.985881,82.21188,145.960311,2.320117e+02,348.168907
1,0.0,0.000000,0.00000,34.985881,8.221188e+01,145.960311
2,0.0,0.000000,0.00000,0.000000,2.842171e-14,34.985881
3,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000
4,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000
5,0.0,0.000000,0.00000,0.000000,0.000000e+00,0.000000


In [8]:
C_tree = pd.DataFrame(np.zeros((T+1)*(T+1)).reshape(T+1, T+1))
C_tree[T] = C_K_tree[T]
C_tree

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,348.168907
1,0.0,0.0,0.0,0.0,0.0,145.960311
2,0.0,0.0,0.0,0.0,0.0,34.985881
3,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.000000


### 시점별 노드별 기대값의 현가 - 무위험이자율로 할인하여 구함

In [9]:
for node in range(T-1, -1,-1):
    for time in range(T-1, -1, -1):
        if time>=node:
            C_tree.loc[node, time] = \
            max(P*C_tree.loc[node, time+1] / (1+Rf) + (1-P)*C_tree.loc[node+1, time+1]/(1+Rf), C_K_tree.loc[node, time])

C_tree

,0,1,2,3,4,5
0,37.042002,61.314001,99.128897,155.853227,237.175520,348.168907
1,0.000000,15.583290,28.249654,50.260325,87.194365,145.960311
2,0.000000,0.000000,4.002886,8.245426,16.984508,34.985881
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
